In [297]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from kmeans import K_Means
# from svd import SVD

In [298]:
data = pd.read_excel('data/Course Recommendation System(1-162).xlsx')
data = data.drop(data.iloc[:, 0:6], axis=1) # Delete unnecessary columns
data_core = data.iloc[:, :5]

courses = list(data.columns)
core_courses = list(data_core.columns)
core_courses

['Data Structures and Algorithms',
 'Computer Architecture\n',
 'Discrete Mathematics\n',
 'Economics',
 'Programming-2']

In [299]:
data = data.fillna(0.5)
X = data.to_numpy()
X_core = data_core.to_numpy()
X_electives = X[:, 5:]

X_core contains our user vectors (ratings for each user in the core courses)

In [300]:
# K-means on user vectors

n_clusters = 5
n_init = 10
kmeans = K_Means(n_clusters, n_init)
kmeans.fit(X_core, 100)
kmeans.get_cluster_size()
cluster_assignment = kmeans.c_assignment

In [301]:
X_comp = np.zeros((1, 25))

for i in range(0, n_clusters):
    temp = X_core[np.where(cluster_assignment == i)]
    mean = np.mean(X_electives[np.where(cluster_assignment == i)], axis=0)
    mean_repeated = np.repeat(mean.reshape(1, -1), temp.shape[0], axis=0)
    X_comp = np.vstack((X_comp, np.concatenate((temp, mean_repeated), axis=1)))
    
X_comp = X_comp[1:, :]

In [302]:
# def QR_Decomposition(A):
#     n, m = A.shape # get the shape of A

#     Q = np.empty((n, n)) # initialize matrix Q
#     u = np.empty((n, n)) # initialize matrix u

#     u[:, 0] = A[:, 0]
#     Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])

#     for i in range(1, n):

#         u[:, i] = A[:, i]
#         for j in range(i):
#             u[:, i] -= (A[:, i] @ Q[:, j]) * Q[:, j] # get each u vector

#         Q[:, i] = u[:, i] / np.linalg.norm(u[:, i]) # compute each e vetor

#     R = np.zeros((n, m))
#     for i in range(n):
#         for j in range(i, m):
#             R[i, j] = A[:, j] @ Q[:, i]

#     return Q, R


# def svd_simultaneous_power_iteration(A, k, epsilon=0.00001):
#     n_orig, m_orig = A.shape
#     if k is None:
#         k=min(n_orig,m_orig)
#     A_orig=A.copy()
#     if n_orig > m_orig:
#         A = A.T @ A
#         n, m = A.shape
#     elif n_orig < m_orig:
#         A = A @ A.T
#         n, m = A.shape
#     else:
#         n,m=n_orig, m_orig
        
#     Q = np.random.rand(n, k)
#     # Q, _ = np.linalg.qr(Q)
#     Q, _ = QR_Decomposition(Q)
#     Q_prev = Q
#     #this part does the block power iteration
#     for i in range(1000):
#         Z = A @ Q
#         # Q, R = np.linalg.qr(Z)
#         Q, R = QR_Decomposition(Z)
#         err = ((Q - Q_prev) ** 2).sum()
#         Q_prev = Q
#         if err < epsilon:
#             break
            
#     singular_values=np.sqrt(np.diag(R))
#     # print(singular_values)
#     # singular_values = singular_values[:-5]
#     # print(singular_values)
#     #deal with different shape input matrices
#     if n_orig < m_orig: 
#         left_vecs=Q.T
#         # left_vecs = left_vecs[:, :-5]
#         print(left_vecs.shape)
#         #use property Values @ V = U.T@A => V=inv(Values)@U.T@A
#         right_vecs=np.linalg.inv(np.diag(singular_values))@left_vecs.T@A_orig
#     elif n_orig==m_orig:
#         left_vecs=Q.T
#         print(left_vecs.shape)
#         right_vecs=left_vecs
#         singular_values=np.square(singular_values)
#     else:
#         right_vecs=Q.T
#         # right_vecs = right_vecs[:-5, :]
#         print(right_vecs.shape)
#         #use property Values @ V = U.T@A => U=A@V@inv(Values)
#         left_vecs=A_orig@ right_vecs.T @np.linalg.inv(np.diag(singular_values))

#     print(left_vecs.shape)
#     print(singular_values.shape)
#     print(right_vecs.shape)
#     return left_vecs, singular_values, right_vecs

In [303]:
import numpy as np

class SVD:
	def __init__(self, A):
		self.u, self.s, self.vh = self.svd(A)

	def gram_schmidt(self, A):
		m, n = A.shape
		Q = np.zeros((m, n))
		R = np.zeros((n, n))

		for j in range(n):
			v = A[:, j]
			for i in range(j):
				R[i, j] = np.dot(Q[:, i], A[:, j])
				v = v - R[i, j] * Q[:, i]
			R[j, j] = np.linalg.norm(v)
			Q[:, j] = v / R[j, j]

		return Q, R

	def qr_eigen(self, A, max_iter=1000, tol=1e-6):

		for _ in range(max_iter):
			Q, R = self.gram_schmidt(A)
			A = np.dot(R, Q)

			if np.amax(np.abs(A)) < tol:
				break

		return np.diag(A), Q

	def svd(self, A):
		m, n = A.shape

		ATA = A.T.dot(A)
		eigenvals, V = self.qr_eigen(ATA)

		print(eigenvals)
		eigenvals = np.array(eigenvals)
		eigenvals[eigenvals<1e-10] = 0
		print(eigenvals)


		idx = eigenvals.argsort()[::-1]
		V = V[:, idx]

		singularvals = np.sqrt(eigenvals[idx])
		S = np.zeros((m, n))
		S[:min(m, n), :min(m, n)] = np.diag(singularvals)

		U = np.zeros((m, m))
		for i in range(min(m, n)):
			if singularvals[i] != 0:
				U[:, i] = A.dot(V[:, i]) / singularvals[i]

		if m >= n:
			return U, S, V.T
		else:
			return V, S.T, U.T

	def return_matrices(self):
		return self.u, self.s, self.vh

In [309]:
X_comp

array([[0.7       , 0.8       , 0.2       , ..., 0.512     , 0.536     ,
        0.52      ],
       [0.65      , 0.7       , 0.2       , ..., 0.512     , 0.536     ,
        0.52      ],
       [0.7       , 0.9       , 0.2       , ..., 0.512     , 0.536     ,
        0.52      ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.39230769, 0.26153846,
        0.54807692],
       [1.        , 0.5       , 0.7       , ..., 0.39230769, 0.26153846,
        0.54807692],
       [0.4       , 0.7       , 0.9       , ..., 0.39230769, 0.26153846,
        0.54807692]])

In [307]:
np.dot(u @ s, vh)

array([[0.7 , 0.8 , 0.2 , ..., 0.  , 0.  , 0.  ],
       [0.65, 0.7 , 0.2 , ..., 0.  , 0.  , 0.  ],
       [0.7 , 0.9 , 0.2 , ..., 0.  , 0.  , 0.  ],
       ...,
       [1.  , 1.  , 1.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 0.5 , 0.7 , ..., 0.  , 0.  , 0.  ],
       [0.4 , 0.7 , 0.9 , ..., 0.  , 0.  , 0.  ]])

In [304]:
SVD = SVD(X_comp)
u, s, vh = SVD.return_matrices()
print(u.shape, s.shape, vh.shape)
print(np.allclose(X_comp, np.dot(u @ s, vh)))

[ 1.28191925e+03  2.72937586e+01  1.93420740e+01  1.48098925e+01
  9.05174324e+00  7.51828789e+00  1.71906073e+00  1.37387299e+00
  5.43379816e-01  3.66541379e-01  1.41357031e-09 -1.17320587e-09
 -2.34540147e-10 -5.61933166e-12 -1.49754065e-12  6.38480475e-13
  6.21372925e-13 -3.92918857e-13  2.94467497e-13  1.97959066e-13
 -1.72776929e-13 -1.41403880e-13  1.38025777e-13  8.15184362e-14
  3.92287211e-14]
[1.28191925e+03 2.72937586e+01 1.93420740e+01 1.48098925e+01
 9.05174324e+00 7.51828789e+00 1.71906073e+00 1.37387299e+00
 5.43379816e-01 3.66541379e-01 1.41357031e-09 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00]
(162, 162) (162, 25) (25, 25)
False


In [305]:
# u, s, vh = np.linalg.svd(X, full_matrices=False)
# print(np.allclose(X, np.dot(u * s, vh)))

In [306]:
# SVD = SVD(X)
# u, s, vh = SVD.return_matrices()
# print(u.shape, s.shape, vh.shape)
# print(np.allclose(X, np.dot(u @ s, vh)))